# 1. Setup

In [ ]:
%conda install geopandas osmnx
%pip install tensorflow
%pip install optree
%pip install --upgrade tensorflow keras
%pip install xgboost
%pip install shap
%pip install numpy


In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np 
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import shapiro, f_oneway
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import calendar
from datetime import datetime, timedelta
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow import keras
from keras.optimizers import Nadam  
from keras.callbacks import EarlyStopping, ReduceLROnPlateau  
import matplotlib.pyplot as plt  
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adamax, Nadam
import optuna

# 2. EDA 

In [ ]:
# Load your dataset
df = pd.read_csv('/home/u894059/.local/dataset/EVChargingStationUsage.csv')

# Separate the 'Start Date' into 'start_date' and 'start_time'
df[['start_date', 'start_time']] = df['Start Date'].str.split(' ', expand=True)

# Optional: Convert 'start_date' to datetime format
df['start_date'] = pd.to_datetime(df['start_date'])

# Define start and end dates
start_year = '01/01/2016'
end_year = '12/31/2020'

# Filter rows for dates between 2016 and 2020
df_filtered = df[(df['start_date'] >= start_year) & (df['start_date'] <= end_year)]

df_filtered.to_csv('/home/u894059/.local/dataset/EVUsage_updated.csv', index=False)


## 2.1 Missing Values

In [ ]:
# Check for missing values in each column
missing_counts = df_filtered.isnull().sum()
print(missing_counts[missing_counts > 0])  # Show only columns with missing values

In [ ]:
missing_percentage = (df_filtered.isnull().sum() / len(df_filtered)) * 100
print(missing_percentage[missing_percentage > 0])  # Show percentage only for columns with missing values

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df_filtered.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.show()

## 2.2 Outliers

In [ ]:
# Loop through each column and calculate Z-scores
for column in df_filtered.select_dtypes(include=[np.number]).columns:  # Only apply to numeric columns
    z_scores = np.abs(stats.zscore(df_filtered[column]))
    outliers = df_filtered[z_scores > 3]
    print(f"Outliers in {column}:\n", outliers)


In [ ]:
# Define a function to detect outliers based on Z-score
def detect_outliers_z(df_filtered, threshold=3):
    z_scores = np.abs(stats.zscore(df_filtered.select_dtypes(include=[np.number])))
    outliers = (z_scores > threshold)
    return df_filtered[~outliers.any(axis=1)]  # Filter rows with no outliers

# Original descriptive statistics
print("Original Data Descriptive Statistics:\n")
print(df_filtered.describe())

# Detect and remove outliers
df_no_outliers = detect_outliers_z(df_filtered)

# Descriptive statistics after removing outliers
print("\nDescriptive Statistics Without Outliers:\n")
print(df_no_outliers.describe())


In [ ]:
# Loop through each numerical column in the DataFrame
for column in df_filtered.select_dtypes(include=[np.number]).columns:
    plt.figure(figsize=(15, 10))
    sns.boxplot(x=df_filtered[column])
    plt.title(f'Box Plot of {column} with Potential Outliers')
    plt.show()

# Histograms for each numeric column
df_filtered.select_dtypes(include=[np.number]).hist(bins=30, figsize=(15, 10))
plt.suptitle('Histograms for Each Numeric Column')
plt.show()


In [ ]:
# Test for normality on original data (with outliers)
for column in df_filtered.select_dtypes(include=[np.number]).columns:
    stat, p = shapiro(df_filtered[column].dropna())  # dropna to ignore NaN values
    print(f'Shapiro-Wilk Test for {column} (with outliers): Statistics={stat:.3f}, p={p:.3f}')

print("\nWithout Outliers:\n")
# Test for normality on data without outliers
for column in df_no_outliers.select_dtypes(include=[np.number]).columns:
    stat, p = shapiro(df_no_outliers[column].dropna())
    print(f'Shapiro-Wilk Test for {column} (without outliers): Statistics={stat:.3f}, p={p:.3f}')


## 2.3 Data Types

In [ ]:
# Print each column name and its data type
for column in df_filtered.columns:
    print(f"{column}: {df_filtered[column].dtype}")


In [ ]:
# Convert the duration columns to timedelta
df_filtered['Total Duration'] = pd.to_timedelta(df_filtered['Total Duration (hh:mm:ss)'])
df_filtered['Charging Time'] = pd.to_timedelta(df_filtered['Charging Time (hh:mm:ss)'])

## 2.4 Univariate Analysis

In [ ]:
# Identify continuous variables
continuous_vars = df_filtered.select_dtypes(include=['float64', 'int64']).columns

# Plot each continuous variable
for var in continuous_vars:
    plt.figure(figsize=(12, 6))
    sns.histplot(df_filtered[var], kde=True, bins=10)
    plt.title(f'Distribution of {var}')
    plt.xlabel(var)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Identify continuous variables
continuous_vars = df_filtered.select_dtypes(include=['float64', 'int64']).columns

# Loop through each continuous variable and calculate summary statistics
for var in continuous_vars:
    summary_stats = df_filtered[var].describe()
    skewness = df_filtered[var].skew()
    kurtosis = df_filtered[var].kurtosis()

    print(f"\nSummary Statistics for {var}:")
    print(summary_stats)

    print(f"\nSkewness for {var}: {skewness:.2f}")
    print(f"Kurtosis for {var}: {kurtosis:.2f}")


In [ ]:
# Identify categorical variables
categorical_vars = ['Station Name', 'MAC Address', 'Org Name', 'Port Type', 'Plug Type', 'Address 1', 'City', 'State/Province', 'Country',
'Currency', 'Ended By', 'County', 'Model Number']  


# Loop through each categorical variable and create a count plot
for var in categorical_vars:
    plt.figure(figsize=(12, 6))
    sns.countplot(x=var, data=df_filtered)
    plt.title(f'Frequency of Categories in {var}')
    plt.xlabel('Categories')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)  # Rotate x-axis labels if needed
    plt.show()


## 2.5 Multivariate Analysis

In [ ]:
# Select only numeric columns
numeric_df = df_filtered.select_dtypes(include=[np.number])

# Calculate correlation matrix
correlation_matrix = numeric_df.corr()

# Visualize the correlation matrix
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


## 2.6 Target Variable Analysis

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(df_filtered['Energy (kWh)'], kde=True, bins=30)
plt.title('Distribution of Target Variable')
plt.xlabel('Target Variable')
plt.ylabel('Frequency')
plt.show()


In [ ]:
summary_stats = df_filtered['Energy (kWh)'].describe()
print("Summary Statistics for Target Variable:")
print(summary_stats)

# Additional metrics: skewness and kurtosis
skewness = df_filtered['Energy (kWh)'].skew()
kurtosis = df_filtered['Energy (kWh)'].kurtosis()

print(f"\nSkewness: {skewness:.2f}")
print(f"Kurtosis: {kurtosis:.2f}")


In [ ]:
# Select only non-numeric columns
non_numeric_columns = df_filtered.select_dtypes(exclude=['number']).columns
print("Non-Numeric Columns:")
print(non_numeric_columns)


In [ ]:
# Calculate the correlation matrix
correlation_matrix = df_filtered.corr()

# Display the correlation of all variables with the target variable
target_correlation = correlation_matrix['Energy (kWh)']
print("Correlation of all variables with the target variable:")
print(target_correlation)


## 2.7 Visualisation

In [ ]:
df_filtered['start_date'] = pd.to_datetime(df_filtered['start_date'])
daily_demand = df_filtered.groupby(df_filtered['start_date'].dt.date)['Energy (kWh)'].sum().reset_index()
daily_demand.columns = ['Date', 'Total Energy (kWh)']

# Create the plot
plt.figure(figsize=(12, 6))
# sns.lineplot(data=daily_demand, x='Date', y='Total Energy (kWh)', marker='o')
sns.lineplot(data=daily_demand, x='Date', y='Total Energy (kWh)')

# Add a vertical line for March 1, 2020
plt.axvline(pd.to_datetime('2020-03-01'), color='red', linestyle='--', label='COVID-19 Pandemic Start')

# Add titles and labels
plt.title('Daily Energy Demand (kWh)')
plt.xlabel('Date')
plt.ylabel('Total Energy (kWh)')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Convert Transaction Date to datetime if you are using it
df_filtered['start_time'] = pd.to_datetime(df_filtered['start_time'])

# Extract the hour from the Transaction Date (or start_date)
df_filtered['Hour'] = df_filtered['start_time'].dt.hour

# Group by hour and calculate the average energy demand
hourly_demand = df_filtered.groupby('Hour')['Energy (kWh)'].mean().reset_index()
hourly_demand.columns = ['Hour', 'Average Energy (kWh)']

# Create the plot
plt.figure(figsize=(12, 6))
sns.barplot(data=hourly_demand, x='Hour', y='Average Energy (kWh)')

# Add titles and labels
plt.title('Average Energy Demand by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Energy (kWh)')
plt.xticks(range(0, 24))  # Set x-ticks for all hours
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
peak_hour = hourly_demand.loc[hourly_demand['Average Energy (kWh)'].idxmax()]
print(f"Peak hour is {peak_hour['Hour']} with an average energy demand of {peak_hour['Average Energy (kWh)']} kWh.")


In [ ]:
df_filtered['start_date'] = pd.to_datetime(df_filtered['start_date'])

# Extract the day of the week (0=Monday, 6=Sunday)
df_filtered['Day of Week'] = df_filtered['start_date'].dt.day_name()

# Group by day of the week and calculate the average energy demand
weekly_demand = df_filtered.groupby('Day of Week')['Energy (kWh)'].mean().reset_index()
weekly_demand.columns = ['Day of Week', 'Average Energy (kWh)']

# Define the order of the days for proper plotting
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_demand['Day of Week'] = pd.Categorical(weekly_demand['Day of Week'], categories=day_order, ordered=True)

# Create the plot
plt.figure(figsize=(12, 6))
sns.barplot(data=weekly_demand, x='Day of Week', y='Average Energy (kWh)')

# Add titles and labels
plt.title('Average Energy Demand by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Energy (kWh)')
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()


In [31]:
df_filtered = pd.read_csv('/Users/yeelam/Documents/Thesis/data/EVUsage_updated.csv')

In [ ]:
# Determine if the day is a weekend or weekday
df_filtered['start_date'] = pd.to_datetime(df_filtered['start_date'])
df_filtered['IsWeekend'] = df_filtered['start_date'].dt.dayofweek >= 5  # Saturday=5, Sunday=6

# Group by weekend/weekday and calculate the average energy demand
weekend_demand = df_filtered.groupby('IsWeekend')['Energy (kWh)'].mean().reset_index()
weekend_demand['IsWeekend'] = weekend_demand['IsWeekend'].map({False: 'Weekday', True: 'Weekend'})
weekend_demand.columns = ['DayType', 'Average Energy (kWh)']

# Create the plot
plt.figure(figsize=(8, 6))
sns.barplot(data=weekend_demand, x='DayType', y='Average Energy (kWh)')

# Add titles and labels
plt.title('Average Energy Demand: Weekday vs Weekend')
plt.xlabel('Day Type')
plt.ylabel('Average Energy (kWh)')
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Group by hour and day type
hourly_daytype_demand = df_filtered.groupby(['Hour', 'IsWeekend'])['Energy (kWh)'].mean().reset_index()
hourly_daytype_demand['IsWeekend'] = hourly_daytype_demand['IsWeekend'].map({False: 'Weekday', True: 'Weekend'})

# Create the plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=hourly_daytype_demand, x='Hour', y='Energy (kWh)', hue='IsWeekend', marker='o')

# Add titles and labels
plt.title('Average Energy Demand by Hour (Weekday vs Weekend)')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Energy (kWh)')
plt.xticks(range(0, 24))
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Create a pivot table for heatmap
heatmap_data = df_filtered.pivot_table(
    index=df_filtered['start_date'].dt.day_name(), 
    columns=df_filtered['Hour'], 
    values='Energy (kWh)', 
    aggfunc='mean'
)

# Sort days of the week
heatmap_data = heatmap_data.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# Plot heatmap
plt.figure(figsize=(14, 7))
sns.heatmap(heatmap_data, cmap="YlGnBu", annot=True, fmt=".1f", cbar_kws={'label': 'Average Energy (kWh)'})

# Add titles and labels
plt.title('Average Energy Demand Heatmap (Hour vs. Day of Week)')
plt.xlabel('Hour of the Day')
plt.ylabel('Day of the Week')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Aggregate data to daily level
daily_data = df_filtered.resample('D', on='start_date')['Energy (kWh)'].sum()

# Perform decomposition
decomposition = seasonal_decompose(daily_data, model='additive', period=365)

# Plot components
decomposition.plot()
plt.suptitle('Seasonality Decomposition of Daily Energy Demand', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
from pandas.plotting import autocorrelation_plot

# Aggregate to daily data
daily_data = df_filtered.resample('D', on='start_date')['Energy (kWh)'].sum()

# Plot auto-correlation
plt.figure(figsize=(10, 6))
autocorrelation_plot(daily_data)
plt.title('Auto-Correlation of Daily Energy Demand')
plt.show()


In [ ]:
# Extract month and hour
df_filtered['start_date'] = pd.to_datetime(df_filtered['start_date'])
df_filtered['Month'] = df_filtered['start_date'].dt.month_name()
hourly_monthly_data = df_filtered.groupby(['Month', 'Hour'])['Energy (kWh)'].mean().reset_index()

# Sort months
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
hourly_monthly_data['Month'] = pd.Categorical(hourly_monthly_data['Month'], categories=month_order, ordered=True)

# Create Facet Grid
g = sns.FacetGrid(hourly_monthly_data, col="Month", col_wrap=4, height=3, aspect=1.5)
g.map(sns.lineplot, 'Hour', 'Energy (kWh)', marker='o')

# Add titles and labels
g.set_titles("{col_name}")
g.set_axis_labels('Hour of the Day', 'Average Energy (kWh)')
g.fig.suptitle('Hourly Trends by Month', y=1.02, fontsize=16)
g.tight_layout()


In [ ]:
# Calculate rolling average
df_filtered['RollingMean'] = df_filtered['Energy (kWh)'].rolling(window=7).mean()

# Plot trends
plt.figure(figsize=(10, 6))
plt.plot(df_filtered['start_date'], df_filtered['Energy (kWh)'], label='Daily Demand', alpha=0.5)
plt.plot(df_filtered['start_date'], df_filtered['RollingMean'], label='7-Day Rolling Average', color='orange')
plt.legend()
plt.title('Rolling Average of Daily Energy Demand')
plt.xlabel('Date')
plt.ylabel('Energy (kWh)')
plt.show()


In [ ]:
# Extract the month
df_filtered['start_date'] = pd.to_datetime(df_filtered['start_date'])
df_filtered['Month'] = df_filtered['start_date'].dt.month_name()

# Group by month and calculate the average energy demand
monthly_demand = df_filtered.groupby('Month')['Energy (kWh)'].mean().reset_index()
monthly_demand.columns = ['Month', 'Average Energy (kWh)']

# Sort months
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_demand['Month'] = pd.Categorical(monthly_demand['Month'], categories=month_order, ordered=True)
monthly_demand = monthly_demand.sort_values('Month')

# Create the plot
plt.figure(figsize=(12, 6))
sns.barplot(data=monthly_demand, x='Month', y='Average Energy (kWh)')

# Add titles and labels
plt.title('Average Energy Demand by Month')
plt.xlabel('Month')
plt.ylabel('Average Energy (kWh)')
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df_filtered = pd.read_csv('/Users/yeelam/Documents/Thesis/data/EVUsage_updated.csv')

# Check the first few rows to understand its structure
print(df_filtered.head())

# Check for missing values and data types
print(df_filtered.info())

# Convert 'start_date' to datetime format
df_filtered['start_date'] = pd.to_datetime(df_filtered['start_date'])  # Convert the 'start_date' column to datetime

# Extract the week number from the 'start_date' column
df_filtered['Week'] = df_filtered['start_date'].dt.isocalendar().week  # Extract the ISO week number
df_filtered['Week'] = df_filtered['Week'].astype(int)


# Now, group by 'Week' and calculate the average energy usage
weekly_demand = df_filtered.groupby('Week').agg({'Energy (kWh)': 'mean'}).reset_index()

# Rename the column for clarity
weekly_demand = weekly_demand.rename(columns={'Energy (kWh)': 'Average Energy (kWh)'})

# Check the cleaned data
print(weekly_demand.head())

# Create the plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=weekly_demand, x='Week', y='Average Energy (kWh)', marker='o')

# Add titles and labels
plt.title('Average Energy Demand by Week')
plt.xlabel('Week')
plt.ylabel('Average Energy (kWh)')
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import folium

# Remove duplicate latitude and longitude pairs
df_filtered_unique = df_filtered.drop_duplicates(subset=['Latitude', 'Longitude'])

# Initialize a map centered around Palo Alto
palo_alto_map = folium.Map(location=[37.4419, -122.1430], zoom_start=12)  # Palo Alto coordinates

# Add charging station locations to the map
for index, row in df_filtered_unique.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Energy (kWh): {row['Energy (kWh)']}"
    ).add_to(palo_alto_map)

# Save the map to an HTML file or display it
palo_alto_map.save("ev_charging_stations_map_unique.html")
palo_alto_map


In [ ]:
# Check the data types
print(weekly_demand.dtypes)

# Inspect the first few rows
print(weekly_demand.head())


In [ ]:
import folium
from folium.plugins import HeatMap

# Assuming you have already filtered your dataframe (df_filtered) to include the relevant columns
# and it contains the columns: 'Latitude', 'Longitude', 'Energy (kWh)'

# Initialize a map centered around Palo Alto
palo_alto_map = folium.Map(location=[37.4419, -122.1430], zoom_start=12)

# Prepare data for HeatMap (latitude, longitude, and optionally, weights like energy usage)
heat_data = df_filtered[['Latitude', 'Longitude', 'Energy (kWh)']].values.tolist()

# Add HeatMap to the map
HeatMap(heat_data, radius=15).add_to(palo_alto_map)

# Save the map to an HTML file
palo_alto_map.save("ev_charging_stations_heatmap.html")

# Display the map (in Jupyter or Python environments that support rendering)
palo_alto_map


In [ ]:
sns.histplot(df_filtered['Energy (kWh)'], bins=30, kde=True)
plt.title('Distribution of Energy Consumption')
plt.xlabel('Energy (kWh)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Group by date and sum energy consumption
daily_consumption = df_filtered.groupby('start_date')['Energy (kWh)'].sum().reset_index()
plt.plot(daily_consumption['start_date'], daily_consumption['Energy (kWh)'])
plt.title('Daily Energy Consumption Over Time')
plt.xlabel('Date')
plt.ylabel('Total Energy (kWh)')
plt.show()


In [ ]:
sns.countplot(x='Port Type', data=df_filtered)
plt.title('Count of Charging Stations by Port Type')
plt.xlabel('Port Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


# 3. Data Processing 

## 3.1 Removing Columns 

In [ ]:
df_filtered = pd.read_csv('/home/u894059/.local/dataset/EVUsage_updated.csv')

In [28]:
base_df = df_filtered.copy(deep=True)

In [ ]:
constant_columns = base_df.columns[base_df.nunique() == 1]

# Print the columns that are going to be dropped
print("Columns with constant values (dropped):", list(constant_columns))

base_final = base_df.copy(deep=True)
base_final = base_final.drop(columns=constant_columns)


In [ ]:
# Check the number of unique values for each column
unique_counts = base_final.nunique()

# Display the number of unique values per column
print(unique_counts)


In [31]:
columns_to_drop = ['Start Date', 'Start Time Zone', 'End Time Zone', 'Currency', 'Total Duration (hh:mm:ss)', 'Charging Time (hh:mm:ss)']
base_final = base_final.drop(columns=columns_to_drop)

# Save the cleaned DataFrame
base_final.to_csv('/home/u894059/.local/dataset/final_dataset_cleaned.csv', index=False)


## 3.2 Imputing Missing Values

In [ ]:
# Separate the 'End Date' and 'Transaction Date (Pacific Time)'
base_final = pd.read_csv('/home/u894059/.local/dataset/final_dataset_cleaned.csv')
base_final[['end_date', 'end_time']] = base_final['End Date'].str.split(' ', expand=True)
base_final[['transaction_date', 'transaction_time']] = base_final['Transaction Date (Pacific Time)'].str.split(' ', expand=True)
base_final['end_date'] = base_final['end_date'].fillna(base_final['transaction_date'])
base_final['end_time'] = base_final['end_time'].fillna(base_final['transaction_time'])
base_final = base_final.drop(columns=['End Date','Transaction Date (Pacific Time)'])

In [40]:
# Impute missing values for transaction_date and transaction_time
base_final['transaction_date'] = base_final['transaction_date'].fillna(base_final['end_date'])
base_final['transaction_time'] = base_final['transaction_time'].fillna(base_final['end_time'])


In [41]:
# Impute missing values for 'EVSE ID' and 'County'
cleaning_df = base_final.copy(deep=True)

In [ ]:
# Create a mapping for 'EVSE ID' and 'County' based on 'Address 1'
evse_mapping = cleaning_df.dropna(subset=['EVSE ID']).groupby('Address 1')['EVSE ID'].first().to_dict()
county_mapping = cleaning_df.dropna(subset=['County']).groupby('Address 1')['County'].first().to_dict()

# Fill missing values in 'EVSE ID' and 'County' using the mapping
cleaning_df['EVSE ID'] = cleaning_df['EVSE ID'].fillna(cleaning_df['Address 1'].map(evse_mapping))
cleaning_df['County'] = cleaning_df['County'].fillna(cleaning_df['Address 1'].map(county_mapping))

print("\nDataFrame after filling missing values:")
print(cleaning_df[['EVSE ID','County','Address 1']].head(10))


In [43]:
# Save the cleaned DataFrame
cleaning_df.to_csv('/home/u894059/.local/dataset/cleaning.csv', index=False)

## 3.3 Converting Categorical Data to Numerical Data

In [ ]:
cleaning_df = pd.read_csv('/home/u894059/.local/dataset/cleaning.csv')
# Print each column name and its data type
for column in cleaning_df.columns:
    print(f"{column}: {cleaning_df[column].dtype}")


In [ ]:
non_numerical_cols = cleaning_df.select_dtypes(exclude=['number']).columns

# Print the non-numerical columns
print("Non-Numerical Columns:")
print(non_numerical_cols)

In [46]:
date_time_columns = ['start_date', 'start_time', 'end_date', 'end_time', 
                     'transaction_date', 'transaction_time']
                     
for column in date_time_columns:
    cleaning_df[column] = pd.to_datetime(cleaning_df[column], errors='coerce')


In [47]:
cleaning_df['Station Name'] = cleaning_df['Station Name'].str.replace('PALO ALTO CA /', '', regex=False).str.strip()

In [ ]:
# Iterate over non-numerical columns and print unique values for each
for col in non_numerical_cols:
    print(f"Unique values for {col}: {cleaning_df[col].unique()}")


In [49]:
cleaning_df = cleaning_df.drop('User ID', axis=1)


In [ ]:
# Remove rows with missing values in 'Ended By' and 'Energy (kWh)' columns
df_clean = cleaning_df.dropna(subset=['Ended By', 'Energy (kWh)'])

# Step 2: Group data by the 'Ended By' categories
groups = [df_clean[df_clean['Ended By'] == category]['Energy (kWh)'] for category in df_clean['Ended By'].unique()]

# Step 3: Perform ANOVA test
f_stat, p_value = stats.f_oneway(*groups)

# Step 4: Output the ANOVA result
print(f"ANOVA p-value for 'Ended By' column: {p_value}")

# Interpretation
if p_value < 0.05:
    print("The 'Ended By' categories have a statistically significant impact on 'Energy (kWh)'.")
else:
    print("The 'Ended By' categories do not have a statistically significant impact on 'Energy (kWh)'.")


In [ ]:
# Specify the columns to be one-hot encoded
covert_columns = ['Port Type', 'Plug Type', 'Ended By', 'County', 'Model Number']

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, dtype=int)

# Perform one-hot encoding on the specified columns
one_hot_encoded = encoder.fit_transform(cleaning_df[covert_columns])

# Convert the result to a DataFrame with appropriate column names
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(covert_columns))

# Concatenate the original DataFrame without the one-hot encoded columns and the one-hot encoded DataFrame
df_one_hot = pd.concat([cleaning_df.drop(columns=covert_columns), one_hot_encoded_df], axis=1)

# Display the DataFrame
print(df_one_hot)


In [ ]:
# Frequency encoding function
def frequency_encode(df, column):
    frequency = df[column].value_counts() / len(df)  # Calculate the frequency
    df[column] = df[column].map(frequency)  # Map the frequencies to the column
    return df

# Apply frequency encoding to specified columns
columns_to_encode = ['Station Name', 'MAC Address', 'Address 1']

for column in columns_to_encode:
    df_one_hot = frequency_encode(df_one_hot, column)

# Check the result
print(df_one_hot[['Station Name', 'MAC Address', 'Address 1']].tail(50))


In [53]:
df_one_hot.to_csv('/home/u894059/.local/dataset/cleaned.csv', index=False)

In [ ]:
# Compute the correlation matrix for all numeric columns
correlations = df_one_hot.corr()

# Set display options to show all rows
pd.set_option('display.max_rows', None)  # None will display all rows

# Display the correlation of all columns with 'Energy (kWh)', sorted in descending order
print(correlations['Energy (kWh)'].sort_values(ascending=False))

# Reset display options to default (optional)
pd.reset_option('display.max_rows')


In [ ]:
# Set a correlation threshold (e.g., 0.1 or -0.1) to remove columns with low correlation
correlation_threshold = 0.03

# Identify columns with correlation below the threshold (including the target column itself)
low_corr_cols = correlations['Energy (kWh)'][abs(correlations['Energy (kWh)']) < correlation_threshold].index

# Drop these columns
df_reduced = df_one_hot.drop(columns=low_corr_cols)

# Display the remaining columns
print("Columns after removing low correlation features:", df_reduced.columns)


In [ ]:
# Adjust the size of the figure (increase even further for better visibility)
plt.figure(figsize=(24, 18))  # Increase the figure size significantly

# Plot the heatmap
sns.heatmap(correlations, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, annot_kws={"size": 10})

# Rotate column and row labels for better visibility
plt.xticks(rotation=90, ha='center')  # Rotate x-axis labels (column names) and align them horizontally
plt.yticks(rotation=0)  # Keep y-axis labels (row names) horizontal

# Set plot title
plt.title('Correlation Matrix of Features', fontsize=18)

# Display the plot
plt.show()


In [ ]:
# Select only numerical columns
X = df_one_hot.select_dtypes(include=['number']).drop(columns=['Energy (kWh)'])
y = df_one_hot['Energy (kWh)']

# Convert data to DMatrix format (XGBoost's optimized data format)
dtrain = xgb.DMatrix(X, label=y, missing=np.nan)

# Set the XGBoost parameters
params = {
    'objective': 'reg:squarederror',  # Use 'reg:squarederror' for regression problems
    'eval_metric': 'rmse',            # Root mean squared error metric
    'max_depth': 6,                   # Maximum depth of the trees
    'learning_rate': 0.1              # Step size for updating weights
}

# Train the XGBoost model
num_boost_round = 100  # Number of boosting rounds (trees)
model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

# Get feature importances
feature_importances = model.get_fscore()

# Convert to a DataFrame and sort by importance score
importance_df = pd.DataFrame(list(feature_importances.items()), columns=["Feature", "Importance"])
importance_df = importance_df.sort_values(by="Importance", ascending=False)

# Display the feature importances
print(importance_df)


In [ ]:
# Plot feature importances as a bar plot
importance_df = importance_df.sort_values(by="Importance", ascending=False)
plt.figure(figsize=(10, 8))
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance")
plt.title("Feature Importances")
plt.gca().invert_yaxis()  # Reverse the y-axis to show the most important features at the top
plt.show()


In [55]:
df_reduced.to_csv('/home/u894059/.local/dataset/reduced.csv', index=False)


# 4. Feature Engineering - Baseline

## 4.1 Temporal Features

In [12]:
df_cleaned = pd.read_csv('/home/u894059/.local/dataset/cleaned.csv', index_col=False)

In [13]:
# Impute 'end_date' with 'transaction_date' where 'end_date' is NaN
df_cleaned['end_date'] = df_cleaned['end_date'].fillna(df_cleaned['start_date'])
df_cleaned['transaction_date'] = df_cleaned['transaction_date'].fillna(df_cleaned['start_date'])

# Impute 'end_time' with 'transaction_time' where 'end_time' is NaN
df_cleaned['end_time'] = df_cleaned['end_time'].fillna(df_cleaned['start_time'])
df_cleaned['transaction_time'] = df_cleaned['transaction_time'].fillna(df_cleaned['start_time'])


In [ ]:
# Temporal Features
# Convert date and time columns to datetime format
df_cleaned['start_date'] = pd.to_datetime(df_cleaned['start_date'], format='%Y/%m/%d')
df_cleaned['end_date'] = pd.to_datetime(df_cleaned['end_date'], format='%Y/%m/%d')
df_cleaned['start_time'] = pd.to_datetime(df_cleaned['start_time'], format='%Y/%m/%d')
df_cleaned['end_time'] = pd.to_datetime(df_cleaned['end_time'], format='%Y/%m/%d')

# Extract date features
df_cleaned['start_year'] = df_cleaned['start_date'].dt.year
df_cleaned['start_month'] = df_cleaned['start_date'].dt.month
df_cleaned['start_day'] = df_cleaned['start_date'].dt.day
df_cleaned['start_weekday'] = df_cleaned['start_date'].dt.dayofweek  # 0=Monday, 6=Sunday

df_cleaned['end_year'] = df_cleaned['end_date'].dt.year
df_cleaned['end_month'] = df_cleaned['end_date'].dt.month
df_cleaned['end_day'] = df_cleaned['end_date'].dt.day
df_cleaned['end_weekday'] = df_cleaned['end_date'].dt.dayofweek

# Extract time features
df_cleaned['start_hour'] = df_cleaned['start_time'].dt.hour
df_cleaned['start_minute'] = df_cleaned['start_time'].dt.minute

df_cleaned['end_hour'] = df_cleaned['end_time'].dt.hour
df_cleaned['end_minute'] = df_cleaned['end_time'].dt.minute

# Calculate transaction duration
df_cleaned['transaction_duration'] = (df_cleaned['end_time'] - df_cleaned['start_time']).dt.total_seconds() / 60  # duration in minutes

# Optional: Define seasonal features based on month
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_cleaned['start_season'] = df_cleaned['start_month'].apply(get_season)
df_cleaned['end_season'] = df_cleaned['end_month'].apply(get_season)

# Display the updated DataFrame with new temporal features
print(df_cleaned)


In [ ]:
df_cleaned['is_weekend'] = df_cleaned['start_weekday'] >= 5
df_cleaned['day_part'] = pd.cut(df_cleaned['start_hour'], bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'], right = False)
print(df_cleaned.isnull().sum())

In [ ]:
pd.set_option('display.max_rows', None)
print(df_cleaned.isnull().sum())

## 4.2 Baseline Features 

In [17]:
cols_to_drop = ['EVSE ID', 'Driver Postal Code','System S/N', 'County_San Mateo County',	'County_Santa Clara County', 'County_nan', 'Model Number_CT2000-HD-CCR', 'Model Number_CT2000-HD-GW1-CCR', 'Model Number_CT2100-HD-CCR', 'Model Number_CT2100-HD-CDMA-CCR', 'Model Number_CT4010-HD-GW', 'Model Number_CT4020-HD', 'Model Number_CT4020-HD-GW', 'Model Number_CTHCR-S', 'Model Number_CTHDR', 'Model Number_CTHDR-S', 'Model Number_nan']
baseline_df = df_cleaned.drop(columns=cols_to_drop, errors='ignore')

In [18]:
baseline_df['hour_sin'] = np.sin(2 * np.pi * baseline_df['start_hour'] / 24)
baseline_df['hour_cos'] = np.cos(2 * np.pi * baseline_df['start_hour'] / 24)
baseline_df['weekday_sin'] = np.sin(2 * np.pi * baseline_df['start_weekday'] / 7)
baseline_df['weekday_cos'] = np.cos(2 * np.pi * baseline_df['start_weekday'] / 7)

In [ ]:
# Convert boolean to integer
baseline_df['is_weekend'] = baseline_df['is_weekend'].astype(int)
baseline_df['transaction_date'] = pd.to_datetime(baseline_df['transaction_date'], format='%Y/%m/%d')
baseline_df['transaction_time'] = pd.to_datetime(baseline_df['transaction_time'], format='%Y/%m/%d %H:%M')
# Extract useful features from 'transaction_date'
baseline_df['transaction_year'] = baseline_df['transaction_date'].dt.year
baseline_df['transaction_month'] = baseline_df['transaction_date'].dt.month
baseline_df['transaction_day'] = baseline_df['transaction_date'].dt.day
baseline_df['transaction_weekday'] = baseline_df['transaction_date'].dt.weekday  # Monday=0, Sunday=6
# Extract useful features from 'transaction_time'
baseline_df['transaction_hour'] = baseline_df['transaction_time'].dt.hour
baseline_df['transaction_minute'] = baseline_df['transaction_time'].dt.minute
baseline_df['transaction_second'] = baseline_df['transaction_time'].dt.second

categorical_columns = ['start_season', 'end_season','day_part']

encoder = OneHotEncoder(sparse=False, dtype=int, handle_unknown='ignore')

# Fit and transform the selected categorical columns
encoded_features = encoder.fit_transform(baseline_df[categorical_columns])

encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))


# Concatenate the encoded columns with the original DataFrame (excluding the original categorical columns)
baseline_df = pd.concat([baseline_df.drop(columns=categorical_columns), encoded_features_df], axis=1)

baseline_df = baseline_df.drop(['start_date', 'start_time', 'end_date', 'end_time', 'transaction_date', 'transaction_time'], axis=1)

# Check for missing values after encoding
print(baseline_df.isnull().sum())


In [21]:
baseline_df.to_csv('/home/u894059/.local/dataset/baseline_df.csv', index=False)

# 5. Additional Features

## 5.1 Holiday Indicators

In [23]:
# List of public holidays
public_holidays = [
    # 2016 Holidays
    '01/01/2016', '01/18/2016', '02/15/2016', '03/31/2016', 
    '05/30/2016', '07/04/2016', '09/05/2016', '11/11/2016', 
    '11/24/2016', '11/25/2016', '12/26/2016',
    
    # 2017 Holidays
    '01/02/2017', '01/16/2017', '02/20/2017', '03/31/2017', 
    '05/29/2017', '07/04/2017', '09/04/2017', '11/10/2017', 
    '11/23/2017', '11/24/2017', '12/25/2017',
    
    # 2018 Holidays
    '01/01/2018', '01/15/2018', '02/19/2018', '03/31/2018', 
    '05/28/2018', '07/04/2018', '09/03/2018', '10/08/2018', 
    '11/12/2018', '11/22/2018', '11/23/2018', '12/25/2018',
    
    # 2019 Holidays
    '01/01/2019', '01/21/2019', '02/18/2019', '04/01/2019', 
    '05/27/2019', '07/04/2019', '09/02/2019', '11/11/2019', 
    '11/28/2019', '11/29/2019', '12/25/2019',
    
    # 2020 Holidays
    '01/01/2020', '01/20/2020', '02/17/2020', '03/31/2020', 
    '05/25/2020', '07/04/2020', '09/07/2020', '11/11/2020', 
    '11/26/2020', '11/27/2020', '12/25/2020'
]

# Convert the list to datetime format
public_holidays = pd.to_datetime(public_holidays)

# Add a new column called 'holiday' and set values based on the public holidays
df_cleaned['holiday'] = df_cleaned['start_date'].isin(public_holidays).astype(int)

# Save the updated DataFrame back to a CSV file
df_cleaned.to_csv('/home/u894059/.local/dataset/EVUsage_updated.csv', index=False)


## 5.2 Weather Features

In [24]:
df_weather = pd.read_csv('/home/u894059/.local/dataset/weather_conditions.csv')

# Separate the 'Start Date' into 'start_date' and 'start_time'
df_weather[['weather_date', 'weather_time']] = df_weather['time'].str.split('T', expand=True)

# Save the updated DataFrame back to a CSV file
df_weather.to_csv('/home/u894059/.local/dataset/weather_updated.csv', index=False)

df_weather['weather_date'] = pd.to_datetime(df_weather['weather_date'])

df_weather['weather_hour'] = pd.to_datetime(df_weather['weather_time'], format='%H:%M').dt.hour

result = pd.merge(df_cleaned, df_weather, left_on=['start_date', 'start_hour'], right_on=['weather_date', 'weather_hour'], how='left')

result.to_csv('/home/u894059/.local/dataset/combined_dataset.csv', index=False)

In [ ]:
columns_to_drop = ['weather_date', 'weather_time', 'weather_hour']
base_final = result.drop(columns=columns_to_drop)

# Save the cleaned DataFrame
base_final.to_csv('/home/u894059/.local/dataset/final_dataset_cleaned.csv', index=False)


## 5.3 Geo Features

In [ ]:
# Assuming your DataFrame is named df
unique_pairs = base_final[['Latitude', 'Longitude']].drop_duplicates()

num_unique_pairs = unique_pairs.shape[0]  # or len(unique_pairs)
print("Unique Pairs:")
print(unique_pairs)
print(f"\nNumber of unique pairs: {num_unique_pairs}")



In [ ]:
# Define a distance in meters
radius = 10000

# Initialize a results list
results = []

# Loop through unique pairs to query OSM
for index, row in unique_pairs.iterrows():
    lat, lon = row['Latitude'], row['Longitude']

    # Create a Point geometry for the location
    point = Point(lon, lat)  # Note: (lon, lat) order for Point

    # Create a GeoDataFrame for the point
    gdf_point = gpd.GeoDataFrame(geometry=[point], crs='EPSG:4326')

    try:
        # Use OSMnx to get amenities within the buffer
        amenities = ox.features_from_point((lat, lon), tags={
            'amenity': ['university','train_station', 'bus_stop', 'taxi', 'bicycle_parking', 'school',
            'college', 'library', 'kindergarten', 'hospital', 'clinic', 'cinema', 'theatre', 'sports_centre', 'stadium', 'park', 'swimming_pool','bank',
            'restaurant', 'cafe', 'bar', 'pub', 'fast_food', 'supermarket', 'convenience', 'department_store', 'bakery', 'shopping_mall', 'post_office',
            'hotel']
        }, dist=radius)

        amenity_types = ['bus_stop', 'taxi', 'train_station', 'university', 'bicycle_parking', 
                 'school', 'college', 'library', 'kindergarten', 'hospital', 'clinic', 
                 'cinema', 'theatre', 'sports_centre', 'stadium', 'park', 'swimming_pool', 
                 'bank', 'restaurant', 'cafe', 'bar', 'pub', 'fast_food', 'supermarket', 
                 'convenience', 'department_store', 'bakery', 'shopping_mall', 
                 'post_office', 'hotel']
        # Dictionary to hold counts
        amenity_counts = {}

        # Count specific amenities
        for amenity in amenity_types:
            amenity_counts[amenity] = amenities[amenities['amenity'] == amenity].shape[0]        

    except Exception as e:
        print(f"Error fetching amenities for ({lat}, {lon}): {e}")
        continue  # Skip to the next iteration

    # Store results for this unique pair
    results.append({
        'Latitude': lat,
        'Longitude': lon,
        **{amenity.replace('_', ' ').title(): count for amenity, count in amenity_counts.items()}
    })

# Create a DataFrame to display the results
results_df = pd.DataFrame(results)

# Print the results
if results_df.empty:
    print("\nNo amenities found for the provided locations.")
else:
    print("\nResults:")
    print(results_df)


In [94]:
results_df.to_csv('/home/u894059/.local/dataset/spatial_dataset.csv', index=False)

In [15]:
results_df = pd.read_csv('/home/u894059/.local/dataset/spatial_dataset.csv')

In [17]:
base_final = pd.read_csv('/home/u894059/.local/dataset/final_dataset_cleaned.csv')

base_final['temp_index'] = base_final.index

# Perform the merge
merged_df = pd.merge(base_final, results_df, on=['Latitude', 'Longitude'])

# Sort by the temporary index to preserve the order of base_final
merged_df = merged_df.sort_values('temp_index').drop(columns=['temp_index'])

# Save the merged DataFrame
merged_df.to_csv('/home/u894059/.local/dataset/final_dataset.csv', index=False)


In [ ]:
zero_columns_mask = (merged_df == 0).all(axis=0)

# Check the boolean mask
print("Columns with all zeros:", zero_columns_mask[zero_columns_mask].index.tolist())

# Filter the DataFrame to keep only columns that are not all zeros
base_cleaned = merged_df.loc[:, ~zero_columns_mask]

base_cleaned = base_cleaned.dropna(axis=1, how='all')

# Display the merged DataFrame
base_cleaned.to_csv('/home/u894059/.local/dataset/base_final.csv', index=False)

## 5.4 Interaction Terms

In [19]:
final_cleaned = pd.read_csv('/home/u894059/.local/dataset/base_final.csv')

In [20]:
cols_to_drop = ['EVSE ID', 'Driver Postal Code','System S/N', 'County_San Mateo County',	'County_Santa Clara County', 'County_nan', 'Model Number_CT2000-HD-CCR', 'Model Number_CT2000-HD-GW1-CCR', 'Model Number_CT2100-HD-CCR', 'Model Number_CT2100-HD-CDMA-CCR', 'Model Number_CT4010-HD-GW', 'Model Number_CT4020-HD', 'Model Number_CT4020-HD-GW', 'Model Number_CTHCR-S', 'Model Number_CTHDR', 'Model Number_CTHDR-S', 'Model Number_nan']
final_reduced = final_cleaned.drop(columns=cols_to_drop, errors='ignore')
final_reduced.to_csv('/home/u894059/.local/dataset/final_reduced.csv', index=False)

In [21]:
final_reduced = pd.read_csv('/home/u894059/.local/dataset/final_reduced.csv')

In [100]:
final_reduced['fee_start_hour'] = final_reduced['Fee'] * final_reduced['start_hour']
final_reduced['start_hour_is_weekend'] = final_reduced['start_hour'] * final_reduced['is_weekend']
final_reduced['start_hour_holiday'] = final_reduced['start_hour'] * final_reduced['holiday']
final_reduced['temperature_hour'] = final_reduced['temperature_2m (°C)'] * final_reduced['start_hour']
final_reduced['rain_hour'] = final_reduced['rain (mm)'] * final_reduced['start_hour']

In [ ]:
# Calculate Pearson correlation between each feature and the target variable
correlations = final_reduced.corr()['Energy (kWh)'].sort_values(ascending=False)

# Set display options to show more rows
pd.set_option('display.max_rows', None)

# Display the correlations again
print(correlations)

## 5.5 Aggregated Features

In [22]:
# Encode hour and day of week as cyclical features
final_reduced['hour_sin'] = np.sin(2 * np.pi * final_reduced['start_hour'] / 24)
final_reduced['hour_cos'] = np.cos(2 * np.pi * final_reduced['start_hour'] / 24)
final_reduced['weekday_sin'] = np.sin(2 * np.pi * final_reduced['start_weekday'] / 7)
final_reduced['weekday_cos'] = np.cos(2 * np.pi * final_reduced['start_weekday'] / 7)

In [ ]:
pd.set_option('display.max_rows', None)
print(final_reduced.dtypes)

In [ ]:
# Convert boolean to integer
final_reduced['is_weekend'] = final_reduced['is_weekend'].astype(int)
final_reduced['transaction_date'] = pd.to_datetime(final_reduced['transaction_date'], format='%Y/%m/%d')
final_reduced['transaction_time'] = pd.to_datetime(final_reduced['transaction_time'], format='%Y/%m/%d %H:%M')
# Extract useful features from 'transaction_date'
final_reduced['transaction_year'] = final_reduced['transaction_date'].dt.year
final_reduced['transaction_month'] = final_reduced['transaction_date'].dt.month
final_reduced['transaction_day'] = final_reduced['transaction_date'].dt.day
final_reduced['transaction_weekday'] = final_reduced['transaction_date'].dt.weekday  # Monday=0, Sunday=6
# Extract useful features from 'transaction_time'
final_reduced['transaction_hour'] = final_reduced['transaction_time'].dt.hour
final_reduced['transaction_minute'] = final_reduced['transaction_time'].dt.minute
final_reduced['transaction_second'] = final_reduced['transaction_time'].dt.second

categorical_columns = ['start_season', 'end_season','day_part']

encoder = OneHotEncoder(sparse=False, dtype=int, handle_unknown='ignore')

# Fit and transform the selected categorical columns
encoded_features = encoder.fit_transform(final_reduced[categorical_columns])

encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))


# Concatenate the encoded columns with the original DataFrame (excluding the original categorical columns)
final_reduced = pd.concat([final_reduced.drop(columns=categorical_columns), encoded_features_df], axis=1)

final_reduced = final_reduced.drop(['time', 'start_date', 'start_time', 'end_date', 'end_time', 'transaction_date', 'transaction_time'], axis=1)

# Check for missing values after encoding
print(final_reduced.isnull().sum())


In [7]:
print(f"Original shape: {final_reduced.shape}")
print(f"Shape after encoding: {encoded_features_df.shape}")


Original shape: (206597, 94)
Shape after encoding: (206597, 12)


In [25]:
final_reduced.to_csv('/home/u894059/.local/dataset/final_reduced2.csv', index=False)

In [ ]:
# Feature Selection
# Prepare data (X and y)
X = final_reduced.drop(columns=['Energy (kWh)'])  # Drop target and non-feature columns
y = final_reduced['Energy (kWh)']

# Train an XGBoost model
model = xgb.XGBRegressor()
model.fit(X, y)

# Use SHAP to interpret feature importance
explainer = shap.Explainer(model)
shap_values = explainer(X)

# Plot feature importance
shap.summary_plot(shap_values, X)


In [ ]:
# Get the feature importance as a dataframe
shap_importance = pd.DataFrame({
    'Feature': X.columns,
    'SHAP Importance': np.abs(shap_values.values).mean(axis=0)  # Mean absolute SHAP value for each feature
})

# Sort features by SHAP importance (descending order)
shap_importance = shap_importance.sort_values(by='SHAP Importance', ascending=False)

# Display the ranked features
print(shap_importance)


In [ ]:
# Select the top 20 features based on SHAP ranking
top_features = shap_importance['Feature'].head(20).values
print(top_features)
# Select the corresponding columns from your dataset
X_selected = final_reduced[top_features]
y = final_reduced['Energy (kWh)']

In [11]:
# Prepare the data using the selected top features
X_selected = final_reduced.drop('Energy (kWh)', axis=1)
y = final_reduced['Energy (kWh)']

# Create sequences for the LSTM model (window size = 30)
def create_sequences(X, y, window_size=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size):
        X_seq.append(X.iloc[i:i+window_size].values)
        y_seq.append(y.iloc[i+window_size])
    return np.array(X_seq), np.array(y_seq)

# Create the sequences
X_seq, y_seq = create_sequences(X_selected, y, window_size=30)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)


# 6. Target Variable Aggregation 

## 6.1.1 Short-term Forecasting (Daily Horizon) - with Additional Features

In [ ]:
# total daily energy consumption

# Load your dataset
# Assuming 'timestamp' is the column with session timestamps and 'Energy (kWh)' is the energy consumption per session
stf = pd.read_csv('/home/u894059/.local/dataset/final_reduced2.csv')

# Group by the 'date' column and sum up the 'Energy (kWh)' for each day
daily_energy = stf.groupby(['start_year', 'start_month', 'start_day'])['Energy (kWh)'].sum().reset_index()

# Rename the aggregated column to make it clear
daily_energy = daily_energy.rename(columns={'Energy (kWh)': 'total_daily_energy_consumption'})

# daily_energy now contains the total energy consumption per day
print(daily_energy.head())


In [27]:
stf = pd.merge(stf, daily_energy, on=['start_year', 'start_month', 'start_day'], how='left')
stf = stf.drop(columns=['Energy (kWh)'])

In [28]:
stf.to_csv('/home/u894059/.local/dataset/short_term.csv', index=False)

## 6.1.2 Short-term Forecasting (Daily Horizon) - with Baseline Features

In [ ]:
# total daily energy consumption

# Load your dataset
# Assuming 'timestamp' is the column with session timestamps and 'Energy (kWh)' is the energy consumption per session
stf_baseline = pd.read_csv('/home/u894059/.local/dataset/baseline_df.csv')

# Group by the 'date' column and sum up the 'Energy (kWh)' for each day
daily_energy_baseline = stf_baseline.groupby(['start_year', 'start_month', 'start_day'])['Energy (kWh)'].sum().reset_index()

# Rename the aggregated column to make it clear
daily_energy_baseline = daily_energy_baseline.rename(columns={'Energy (kWh)': 'total_daily_energy_baseline_consumption'})

# daily_energy_baseline now contains the total energy consumption per day
print(daily_energy_baseline.head())

stf_baseline = pd.merge(stf_baseline, daily_energy_baseline, on=['start_year', 'start_month', 'start_day'], how='left')
stf_baseline = stf_baseline.drop(columns=['Energy (kWh)'])


In [30]:
stf_baseline.to_csv('/home/u894059/.local/dataset/short_term_baseline.csv', index=False)

## 6.2.1 Mid-term Forecasting (Weekly Horizon) - with Additional Features

In [ ]:
# total weekly energy consumption

# Load your dataset
# Assuming 'timestamp' is the column with session timestamps and 'Energy (kWh)' is the energy consumption per session
mtf = pd.read_csv('/home/u894059/.local/dataset/final_reduced2.csv')

# Create a date column from start_year, start_month, and start_day
mtf['date'] = pd.to_datetime(mtf['start_year'].astype(str) + '-' + 
                             mtf['start_month'].astype(str) + '-' + 
                             mtf['start_day'].astype(str), errors='coerce')


# Extract the ISO week number from the new date column
mtf['week'] = mtf['date'].dt.week  # Standard week numbering without ISO adjustments

mtf.loc[(mtf['week'] == 53) & (mtf['date'].dt.month == 1), 'week'] = 1

# Aggregate energy by year, month, and week
weekly_energy = mtf.groupby(['start_year', 'start_month', 'week'])['Energy (kWh)'].sum().reset_index()
weekly_energy.rename(columns={'Energy (kWh)': 'total_weekly_energy_consumption'}, inplace=True)

# Merge weekly totals back to your main DataFrame
mtf = pd.merge(mtf, weekly_energy, on=['start_year', 'start_month', 'week'], how='left').drop(columns=['Energy (kWh)', 'date'])
mtf = mtf.drop(columns=['week'])
print(mtf.head())



In [121]:
mtf.to_csv('/home/u894059/.local/dataset/mid_term.csv', index=False)

## 6.2.2 Mid-term Forecasting (Weekly Horizon) - with Baseline Features

In [ ]:
# total weekly energy consumption

# Load your dataset
# Assuming 'timestamp' is the column with session timestamps and 'Energy (kWh)' is the energy consumption per session
mtf_baseline = pd.read_csv('/home/u894059/.local/dataset/baseline_df.csv')

# Create a date column from start_year, start_month, and start_day
mtf_baseline['date'] = pd.to_datetime(mtf_baseline['start_year'].astype(str) + '-' + 
                             mtf_baseline['start_month'].astype(str) + '-' + 
                             mtf_baseline['start_day'].astype(str), errors='coerce')


# Extract the ISO week number from the new date column
mtf_baseline['week'] = mtf_baseline['date'].dt.week  # Standard week numbering without ISO adjustments

mtf_baseline.loc[(mtf_baseline['week'] == 53) & (mtf_baseline['date'].dt.month == 1), 'week'] = 1

# Aggregate energy by year, month, and week
weekly_energy = mtf_baseline.groupby(['start_year', 'start_month', 'week'])['Energy (kWh)'].sum().reset_index()
weekly_energy.rename(columns={'Energy (kWh)': 'total_weekly_energy_consumption'}, inplace=True)

# Merge weekly totals back to your main DataFrame
mtf_baseline = pd.merge(mtf_baseline, weekly_energy, on=['start_year', 'start_month', 'week'], how='left').drop(columns=['Energy (kWh)', 'date'])

print(mtf_baseline.head())



In [3]:
mtf_baseline.to_csv('/home/u894059/.local/dataset/mid_term_baseline.csv', index=False)

# 7. Input Features Aggregation

## 7.1 Hourly Aggregated Features - Short Term

In [85]:
stf_haf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')


In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
print(stf_haf.head())  # Display the first few rows


In [ ]:
# Define columns for which we want sum and mean aggregation
columns_to_sum = ['GHG Savings (kg)', 'Gasoline Savings (gallons)', 'transaction_duration']
columns_to_mean = ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'rain (mm)', 
                   'snowfall (cm)', 'cloud_cover (%)', 'wind_speed_100m (km/h)']

input_features = stf_haf[['start_year', 'start_month', 'start_day', 'start_hour'] + columns_to_sum + columns_to_mean]

# Aggregate the columns separately for sum and mean
hourly_features_sum = input_features.groupby(
    ['start_year', 'start_month', 'start_day', 'start_hour']
)[columns_to_sum].sum().reset_index()

hourly_features_mean = input_features.groupby(
    ['start_year', 'start_month', 'start_day', 'start_hour']
)[columns_to_mean].mean().reset_index()

# Merge the sum and mean dataframes
hourly_features = pd.merge(hourly_features_sum, hourly_features_mean, 
                            on=['start_year', 'start_month', 'start_day', 'start_hour'])

print(hourly_features.head())

In [ ]:
# Drop the common columns from stf_haf before merging (to avoid duplication)
stf_haf_without_common_columns = stf_haf.drop(columns=columns_to_sum + columns_to_mean)

# Merge the hourly_features with stf_haf (without the common columns)
stf_haf = pd.merge(stf_haf_without_common_columns, hourly_features, 
                   on=['start_year', 'start_month', 'start_day', 'start_hour'], 
                   how='left', 
                   suffixes=('_original', '_aggregated'))

# Display the updated DataFrame
print(stf_haf.head())

# Optionally, save the result to a new CSV file
stf_haf.to_csv('/home/u894059/.local/dataset/short_term_haf.csv', index=False)

## 7.2.1 Daily Aggregated Features - Short Term

In [99]:
stf_daf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

In [ ]:
# Define columns for which we want sum and mean aggregation
columns_to_sum = ['GHG Savings (kg)', 'Gasoline Savings (gallons)', 'transaction_duration']
columns_to_mean = ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'rain (mm)', 
                   'snowfall (cm)', 'cloud_cover (%)', 'wind_speed_100m (km/h)']

input_features = stf_daf[['start_year', 'start_month', 'start_day'] + columns_to_sum + columns_to_mean]

# Aggregate the columns separately for sum and mean
daily_features_sum = input_features.groupby(
    ['start_year', 'start_month', 'start_day']
)[columns_to_sum].sum().reset_index()

daily_features_mean = input_features.groupby(
    ['start_year', 'start_month', 'start_day']
)[columns_to_mean].mean().reset_index()

# Merge the sum and mean dataframes
daily_features = pd.merge(daily_features_sum, daily_features_mean, 
                            on=['start_year', 'start_month', 'start_day'])

print(daily_features.head())

In [ ]:
# Drop the common columns from stf_daf before merging (to avoid duplication)
stf_daf_without_common_columns = stf_daf.drop(columns=columns_to_sum + columns_to_mean)

# Merge the daily_features with stf_daf (without the common columns)
stf_daf = pd.merge(stf_daf_without_common_columns, daily_features, 
                   on=['start_year', 'start_month', 'start_day'], 
                   how='left', 
                   suffixes=('_original', '_aggregated'))

# Display the updated DataFrame
print(stf_daf.head())

# Optionally, save the result to a new CSV file
stf_daf.to_csv('/home/u894059/.local/dataset/short_term_daf.csv', index=False)

## 7.2.2 Daily Aggregated Features - Mid Term

In [124]:
mtf_daf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

In [ ]:
# Define columns for which we want sum and mean aggregation
columns_to_sum = ['GHG Savings (kg)', 'Gasoline Savings (gallons)', 'transaction_duration']
columns_to_mean = ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'rain (mm)', 
                   'snowfall (cm)', 'cloud_cover (%)', 'wind_speed_100m (km/h)']

input_features = mtf_daf[['start_year', 'start_month', 'start_day'] + columns_to_sum + columns_to_mean]

# Aggregate the columns separately for sum and mean
daily_features_sum = input_features.groupby(
    ['start_year', 'start_month', 'start_day']
)[columns_to_sum].sum().reset_index()

daily_features_mean = input_features.groupby(
    ['start_year', 'start_month', 'start_day']
)[columns_to_mean].mean().reset_index()

# Merge the sum and mean dataframes
daily_features = pd.merge(daily_features_sum, daily_features_mean, 
                            on=['start_year', 'start_month', 'start_day'])

print(daily_features.head())

In [ ]:
# Drop the common columns from mtf_daf before merging (to avoid duplication)
mtf_daf_without_common_columns = mtf_daf.drop(columns=columns_to_sum + columns_to_mean)

# Merge the daily_features with mtf_daf (without the common columns)
mtf_daf = pd.merge(mtf_daf_without_common_columns, daily_features, 
                   on=['start_year', 'start_month', 'start_day'], 
                   how='left', 
                   suffixes=('_original', '_aggregated'))

# Display the updated DataFrame
print(mtf_daf.head())

# Optionally, save the result to a new CSV file
mtf_daf.to_csv('/home/u894059/.local/dataset/mid_term_daf.csv', index=False)

## 7.3 Weekly Aggregated Features - Mid Term

In [4]:
mtf_waf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

In [ ]:
# Define columns for which we want sum and mean aggregation
columns_to_sum = ['GHG Savings (kg)', 'Gasoline Savings (gallons)', 'transaction_duration']
columns_to_mean = ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'rain (mm)', 
                   'snowfall (cm)', 'cloud_cover (%)', 'wind_speed_100m (km/h)']

# Create a date column from start_year, start_month, and start_day
mtf_waf['date'] = pd.to_datetime(mtf_waf['start_year'].astype(str) + '-' + 
                             mtf_waf['start_month'].astype(str) + '-' + 
                             mtf_waf['start_day'].astype(str), errors='coerce')


# Extract the ISO week number from the new date column
mtf_waf['week'] = mtf_waf['date'].dt.week  # Standard week numbering without ISO adjustments

mtf_waf.loc[(mtf_waf['week'] == 53) & (mtf_waf['date'].dt.month == 1), 'week'] = 1

# Define the columns you want to aggregate
input_features = mtf_waf[['start_year', 'start_month', 'start_day', 'week'] + columns_to_sum + columns_to_mean]

# Aggregate the columns separately for sum and mean
weekly_features_sum = input_features.groupby(['start_year', 'start_month', 'week'])[columns_to_sum].sum().reset_index()
weekly_features_mean = input_features.groupby(['start_year', 'start_month', 'week'])[columns_to_mean].mean().reset_index()

# Merge the sum and mean dataframes
weekly_features = pd.merge(weekly_features_sum, weekly_features_mean, on=['start_year', 'start_month', 'week'])

# # Drop the week column if not needed in the final output
# weekly_features = weekly_features.drop(columns='week')

# Display the first few rows of the resulting DataFrame
print(weekly_features.head())


In [ ]:
# Drop the common columns from mtf_waf before merging (to avoid duplication)
mtf_waf_without_common_columns = mtf_waf.drop(columns=columns_to_sum + columns_to_mean)

# Merge the daily_features with mtf_waf (without the common columns)
mtf_waf = pd.merge(mtf_waf_without_common_columns, weekly_features, 
                   on=['start_year', 'start_month', 'week'], 
                   how='left', 
                   suffixes=('_original', '_aggregated'))
                   
mtf_waf = mtf_waf.drop(columns=['week', 'date'])

# Display the updated DataFrame
print(mtf_waf.head())

# Optionally, save the result to a new CSV file
mtf_waf.to_csv('/home/u894059/.local/dataset/mid_term_waf.csv', index=False)

# 8. Models - Additional Features

## 8.1.1 LSTM Model - Short Term 

In [69]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    LSTM(64, return_sequences=False),  # Second LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot1.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 8.1.2 LSTM Model - Mid Term 

In [21]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 128 units
    Dropout(0.2),  # Dropout rate set to 0.2
    LSTM(16, return_sequences=False),  # Second LSTM layer with 16 units
    Dropout(0.2),  # Dropout rate set to 0.2
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot2.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 8.2.1 Transformer - Short Term 

In [28]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=128,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict([X_test_seq, X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 8.2.2 Transformer - Mid Term 

In [35]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=128,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict([X_test_seq, X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 8.3.1 Hybrid Model - Short Term 

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 8.3.2 Hybrid Model - Mid Term

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


# 9. Models - Baseline Features

## 9.1.1 LSTM - Short Term 

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_baseline.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_baseline_consumption'])
target = stf['total_daily_energy_baseline_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    LSTM(64, return_sequences=False),  # Second LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot1.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 9.1.2 LSTM - Mid Term 

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term_baseline.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 128 units
    Dropout(0.2),  # Dropout rate set to 0.2
    LSTM(16, return_sequences=False),  # Second LSTM layer with 16 units
    Dropout(0.2),  # Dropout rate set to 0.2
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot2.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 9.2.1 Transformer - Short Term 

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_baseline.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_baseline_consumption'])
target = stf['total_daily_energy_baseline_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=128,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict([X_test_seq, X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 9.2.2 Transformer - Mid Term 

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/mid_term_baseline.csv')

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=128,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict([X_test_seq, X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 9.3.1 Hybrid Model - Short Term 

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_baseline.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_baseline_consumption'])
target = stf['total_daily_energy_baseline_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 9.3.2 Hybrid Model - Mid Term 

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term_baseline.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_baseline_consumption'])
target = mtf['total_weekly_energy_baseline_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


# 10. Models - Short Term Granularity

## 10.1.1 LSTM - Hourly 

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_haf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    LSTM(64, return_sequences=False),  # Second LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot1.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 10.1.2 LSTM - Daily

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_daf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    LSTM(64, return_sequences=False),  # Second LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot2.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 10.2.1 Transformer - Hourly 

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_haf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define Transformer Encoder block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate):
    # Multi-head self-attention layer
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    # Add & Normalize
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    # Feed-forward layer
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    # Add & Normalize
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define the Input Layer for Transformer (3D input: samples, time_steps, features)
inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # Updated input shape to use X_train_seq

# Add a transformer encoder block
x = transformer_encoder(inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)
x = transformer_encoder(inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)
x = transformer_encoder(inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(x)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = Model(inputs, outputs)

# Compile the model
initial_learning_rate = 0.0001  # You can change this value

# Compile the model with the specified learning rate
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore the best weights when stopping
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6, 
    verbose=1
)

# Train the model with the early stopping callback
history = model.fit(
    X_train_seq, y_train_seq,  # Use X_train_seq and y_train_seq
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use X_val_seq and y_val_seq
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Evaluate the model on the test set
test_loss = model.evaluate(X_test_seq, y_test_seq)  # Use X_test_seq and y_test_seq for evaluation
print(f"Test Loss: {test_loss}")

# Make predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Use X_test_seq for prediction

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)  # Inverse transform predictions to original scale
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))  # Inverse transform test data to original scale

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot1.png'

# Save the plot to the specified path
plt.savefig(save_path)

plt.show()


## 10.2.2 Transformer - Daily

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_daf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define Transformer Encoder block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate):
    # Multi-head self-attention layer
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    # Add & Normalize
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    # Feed-forward layer
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    # Add & Normalize
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define the Input Layer for Transformer (3D input: samples, time_steps, features)
inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # Updated input shape to use X_train_seq

# Add a transformer encoder block
x = transformer_encoder(inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)
x = transformer_encoder(inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)
x = transformer_encoder(inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(x)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = Model(inputs, outputs)

# Compile the model
initial_learning_rate = 0.0001  # You can change this value

# Compile the model with the specified learning rate
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore the best weights when stopping
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6, 
    verbose=1
)

# Train the model with the early stopping callback
history = model.fit(
    X_train_seq, y_train_seq,  # Use X_train_seq and y_train_seq
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use X_val_seq and y_val_seq
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Evaluate the model on the test set
test_loss = model.evaluate(X_test_seq, y_test_seq)  # Use X_test_seq and y_test_seq for evaluation
print(f"Test Loss: {test_loss}")

# Make predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Use X_test_seq for prediction

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)  # Inverse transform predictions to original scale
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))  # Inverse transform test data to original scale

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot2.png'

# Save the plot to the specified path
plt.savefig(save_path)

plt.show()


## 10.3.1 Hybrid Model - Hourly 

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_haf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 10.3.2 Hybrid Model - Daily

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term_daf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


# 11. Models - Mid Term Granularity

## 11.1.1 LSTM - Daily

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/mid_term_daf.csv')
stf = stf.drop(columns=['date', 'week'])

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    LSTM(64, return_sequences=False),  # Second LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot1.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 11.1.2 LSTM - Weekly

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/mid_term_waf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    LSTM(64, return_sequences=False),  # Second LSTM layer with 64 units
    Dropout(0.3),  # Dropout rate set to 0.3
    Dense(1)  # Final output layer for regression (predicting continuous values)
])

# Compile the model with a low learning rate for stable training
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model using the sequences
history = model.fit(
    X_train_seq, y_train_seq,  # Use sequences for training
    epochs=50, 
    batch_size=128, 
    validation_data=(X_val_seq, y_val_seq),  # Use sequences for validation
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set (scaled values)
y_pred_scaled = model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Define the path where you want to save the plot
save_path = '/home/u894059/.local/plot2.png'

# Evaluate the model on the scaled test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 11.2.1 Transformer - Daily

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/mid_term_daf.csv')
stf = stf.drop(columns = ['date', 'week'])

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=128,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict([X_test_seq, X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 11.2.2 Transformer - Weekly

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/mid_term_waf.csv')

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=10)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=10)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=10)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=32, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Create the model
model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=128,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict([X_test_seq, X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot2.png')
plt.show()


## 11.3.1 Hybrid Model - Daily

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term_daf.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 11.3.2 Hybrid Model - Weekly

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term_waf.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions on the test set
y_pred_scaled = model.predict(X_test_seq)  # Fixing this line

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 30
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Example Hyperparameters
lstm_units = 128
head_size = 64
num_heads = 8
ff_dim = 256
num_blocks = 3
dropout_rate = 0.2
input_shape = (sequence_length, features.shape[1])

model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='mse', 
              metrics=['mae'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=100,
    batch_size=32,  # Experiment with batch size if needed
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
y_pred_scaled = model.predict(X_test_seq)
mse = mean_squared_error(y_test_seq, y_pred_scaled)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_seq, y_pred_scaled)

# Inverse transform predictions
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}")

# Plot predictions vs actual values

plt.figure(figsize=(10, 6))
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title("Flood Forecasting: Actual vs Predicted")
plt.xlabel("Time Steps")
plt.ylabel("Runoff")
plt.show()


# 12. Hyperparameter Tuning

## 12.1.1 LSTM Hyperparameters - Short Term

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter tuning
    lstm_units = trial.suggest_int('lstm_units', 32, 128)  # LSTM units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)  # Dropout rate range
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)  
  # Log scale for learning rate
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size options

    # Build the model with the suggested hyperparameters
    model = Sequential([
        Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # Input layer
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units, return_sequences=False),
        Dropout(dropout_rate),
        Dense(1)])

    model.compile(optimizer=Nadam(learning_rate=learning_rate), loss='mse')

    # Define early stopping and learning rate reduction callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train the model
    model.fit(
        X_train_seq, y_train_seq, 
        epochs=30, 
        batch_size=batch_size, 
        validation_data=(X_val_seq, y_val_seq), 
        callbacks=[early_stopping, reduce_lr], 
        verbose=1  # Suppress training output
    )

    # Get predictions and calculate the MSE on the validation set
    y_pred_scaled = model.predict(X_val_seq)
    mse = mean_squared_error(y_val_seq, y_pred_scaled)
    return mse  # We want to minimize the MSE

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Perform 50 trials

# Print the best hyperparameters found
print("Best hyperparameters: ", study.best_params)

# Use the best hyperparameters to train the final model
best_params = study.best_params

# Final model definition with the best hyperparameters
final_model = Sequential([
    LSTM(best_params['lstm_units'], return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  
    Dropout(best_params['dropout_rate']),  
    LSTM(best_params['lstm_units'], return_sequences=False),  
    Dropout(best_params['dropout_rate']),  
    Dense(1)
])

final_model.compile(optimizer=Nadam(learning_rate=best_params['learning_rate']), loss='mse')

# Train the final model
final_model.fit(
    X_train_seq, y_train_seq, 
    epochs=50, 
    batch_size=best_params['batch_size'], 
    validation_data=(X_val_seq, y_val_seq),
    verbose=1
)

# Make predictions on the test set (scaled values)
y_pred_scaled = final_model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Plot predictions vs actual values
save_path = '/home/u894059/.local/LSTM_STG_hourly.png'

# Evaluate the model on the scaled test set
test_loss = final_model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = final_model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 12.1.2 LSTM Hyperparameters - Mid Term

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter tuning
    lstm_units = trial.suggest_int('lstm_units', 32, 128)  # LSTM units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)  # Dropout rate range
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)  
  # Log scale for learning rate
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size options

    # Build the model with the suggested hyperparameters
    model = Sequential([
        Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  # Input layer
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units, return_sequences=False),
        Dropout(dropout_rate),
        Dense(1)])

    model.compile(optimizer=Nadam(learning_rate=learning_rate), loss='mse')

    # Define early stopping and learning rate reduction callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train the model
    model.fit(
        X_train_seq, y_train_seq, 
        epochs=30, 
        batch_size=batch_size, 
        validation_data=(X_val_seq, y_val_seq), 
        callbacks=[early_stopping, reduce_lr], 
        verbose=1  # Suppress training output
    )

    # Get predictions and calculate the MSE on the validation set
    y_pred_scaled = model.predict(X_val_seq)
    mse = mean_squared_error(y_val_seq, y_pred_scaled)
    return mse  # We want to minimize the MSE

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Perform 50 trials

# Print the best hyperparameters found
print("Best hyperparameters: ", study.best_params)

# Use the best hyperparameters to train the final model
best_params = study.best_params

# Final model definition with the best hyperparameters
final_model = Sequential([
    LSTM(best_params['lstm_units'], return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),  
    Dropout(best_params['dropout_rate']),  
    LSTM(best_params['lstm_units'], return_sequences=False),  
    Dropout(best_params['dropout_rate']),  
    Dense(1)
])

final_model.compile(optimizer=Nadam(learning_rate=best_params['learning_rate']), loss='mse')

# Train the final model
final_model.fit(
    X_train_seq, y_train_seq, 
    epochs=50, 
    batch_size=best_params['batch_size'], 
    validation_data=(X_val_seq, y_val_seq),
    verbose=1
)

# Make predictions on the test set (scaled values)
y_pred_scaled = final_model.predict(X_test_seq)

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Plot predictions vs actual values
save_path = '/home/u894059/.local/LSTM_STG_hourly.png'

# Evaluate the model on the scaled test set
test_loss = final_model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")

# Make predictions on the scaled test set
predictions_scaled = final_model.predict(X_test_seq)

# Optionally, inverse transform predictions and actual values to the original scale
predictions = target_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the scaled values (0-1 range) or original values
plt.plot(y_test_original, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')

# Save the plot to the specified path
plt.savefig(save_path)

# Show the plot
plt.show()


## 12.2.1 LSTM Optimiser - Short Term

In [ ]:
# List of optimizers to test
optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop,
    "adagrad": Adagrad,
    "adamax": Adamax,
    "nadam": Nadam
}

# Dictionary to store results
results = {}

# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Iterate over each optimizer
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    print(f"Using optimizer: {opt_name}")
    
    # Define the LSTM model
    model = Sequential([
        keras.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        LSTM(109, return_sequences=True),
        Dropout(0.2053906727416861),  
        LSTM(109, return_sequences=False),  
        Dropout(0.2053906727416861),  
        Dense(1)  
    ])

    # Compile the model with the current optimizer
    model.compile(optimizer=opt_name, loss='mse')
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
    
    # Train the model using the sequences
    history = model.fit(
        X_train_seq, y_train_seq,  
        epochs=50, 
        batch_size=32, 
        validation_data=(X_val_seq, y_val_seq),  
        callbacks=[early_stopping, reduce_lr],
        verbose=1  # Suppress detailed logs for readability
    )
    
    # Evaluate the model on the test set
    test_loss = model.evaluate(X_test_seq, y_test_seq, verbose=1)  # MSE on test data
    
    # Get predictions and calculate metrics
    predictions_scaled = model.predict(X_test_seq)
    mse_scaled = mean_squared_error(y_test_seq, predictions_scaled)
    rmse_scaled = np.sqrt(mse_scaled)
    mae_scaled = mean_absolute_error(y_test_seq, predictions_scaled)
    
    # Store results
    results[opt_name] = {
        "Test Loss (MSE)": test_loss,
        "RMSE (scaled)": rmse_scaled,
        "MAE (scaled)": mae_scaled
    }

# Display results for comparison
print("\nOptimizer Performance Comparison:")
for opt_name, metrics in results.items():
    print(f"{opt_name}: {metrics}")

# Example Plot (Using the best optimizer's predictions)
best_optimizer = min(results, key=lambda k: results[k]["RMSE (scaled)"])
print(f"\nBest optimizer based on RMSE: {best_optimizer}")

# Re-train and visualize using the best optimizer
model.compile(optimizer=best_optimizer, loss='mse')
model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)
final_predictions_scaled = model.predict(X_test_seq)
final_predictions = target_scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

plt.plot(y_test_original, label='Actual')
plt.plot(final_predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot_best_optimizer.png')
plt.show()


## 12.2.2 LSTM Optimiser - Mid Term

In [ ]:
# List of optimizers to test
optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop,
    "adagrad": Adagrad,
    "adamax": Adamax,
    "nadam": Nadam
}

# Dictionary to store results
results = {}

# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)  # Use transform, not fit_transform
X_test_scaled = feature_scaler.transform(X_test)  # Use transform, not fit_transform

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()  # Use transform, not fit_transform
y_test_scaled = target_scaler.transform(y_test).flatten()  # Use transform, not fit_transform

# Convert the scaled features to sequences (after scaling)
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Iterate over each optimizer
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    print(f"Using optimizer: {opt_name}")
    
    # Define the LSTM model
    model = Sequential([
        keras.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        LSTM(126, return_sequences=True),
        Dropout(0.4449093799784903),  
        LSTM(126, return_sequences=False),  
        Dropout(0.4449093799784903),  
        Dense(1)  
    ])
 
    # Compile the model with the current optimizer
    model.compile(optimizer=opt_name, loss='mse')
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
    
    # Train the model using the sequences
    history = model.fit(
        X_train_seq, y_train_seq,  
        epochs=50, 
        batch_size=128, 
        validation_data=(X_val_seq, y_val_seq),  
        callbacks=[early_stopping, reduce_lr],
        verbose=1  # Suppress detailed logs for readability
    )
    
    # Evaluate the model on the test set
    test_loss = model.evaluate(X_test_seq, y_test_seq, verbose=1)  # MSE on test data
    
    # Get predictions and calculate metrics
    predictions_scaled = model.predict(X_test_seq)
    mse_scaled = mean_squared_error(y_test_seq, predictions_scaled)
    rmse_scaled = np.sqrt(mse_scaled)
    mae_scaled = mean_absolute_error(y_test_seq, predictions_scaled)
    
    # Store results
    results[opt_name] = {
        "Test Loss (MSE)": test_loss,
        "RMSE (scaled)": rmse_scaled,
        "MAE (scaled)": mae_scaled
    }

# Display results for comparison
print("\nOptimizer Performance Comparison:")
for opt_name, metrics in results.items():
    print(f"{opt_name}: {metrics}")

# Example Plot (Using the best optimizer's predictions)
best_optimizer = min(results, key=lambda k: results[k]["RMSE (scaled)"])
print(f"\nBest optimizer based on RMSE: {best_optimizer}")

# Re-train and visualize using the best optimizer
model.compile(optimizer=best_optimizer, loss='mse')
model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)
final_predictions_scaled = model.predict(X_test_seq)
final_predictions = target_scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

plt.plot(y_test_original, label='Actual')
plt.plot(final_predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot_best_optimizer.png')
plt.show()


## 12.3.1 Transformer Hyperparameters - Short Term

In [ ]:
# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the function to build and compile the model (for use with Optuna)
def build_model(trial):
    # Sample hyperparameters from Optuna trial
    head_size = trial.suggest_int('head_size', 16, 64)
    num_heads = trial.suggest_int('num_heads', 2, 8)
    ff_dim = trial.suggest_int('ff_dim', 32, 128)
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    
    encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
    encoder_outputs = transformer_encoder(encoder_inputs, head_size, num_heads, ff_dim, dropout_rate,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

    # Decoder also uses the same sequence data for simplicity
    decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
    decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate)

    x = layers.GlobalAveragePooling1D()(encoder_outputs)
    outputs = layers.Dense(1)(x)

    model = keras.Model(inputs=encoder_inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

# Modify the `objective` function to use the correct data structure
def objective(trial):
    model = build_model(trial)

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train the model
    model.fit(X_train_seq, y_train_seq, epochs=30, batch_size=128,
          validation_data=(X_val_seq, y_val_seq),
          callbacks=[early_stopping, reduce_lr], verbose=1)

    # Get predictions on the validation set
    y_pred_scaled = model.predict([X_val_seq])

    # Calculate metrics on the scaled values
    mse_scaled = mean_squared_error(y_val_seq, y_pred_scaled)
    rmse_scaled = np.sqrt(mse_scaled)

    return rmse_scaled

# Create an Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Get the best trial and print the best hyperparameters
print(f"Best trial: {study.best_trial.number}")
print(f"Best hyperparameters: {study.best_trial.params}")

# Build the model using the best hyperparameters
best_model = build_model(study.best_trial)

early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Retrain the model with the best hyperparameters
best_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=128,
               validation_data=(X_val_seq, y_val_seq),
               callbacks=[early_stopping, reduce_lr])

# Get predictions on the test set
y_pred_scaled = best_model.predict([X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()



## 12.3.2 Transformer Hyperparameters - Mid Term

In [ ]:
# Load the dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the function to build and compile the model (for use with Optuna)
def build_model(trial):
    # Sample hyperparameters from Optuna trial
    head_size = trial.suggest_int('head_size', 16, 64)
    num_heads = trial.suggest_int('num_heads', 2, 8)
    ff_dim = trial.suggest_int('ff_dim', 32, 128)
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 128)

    encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
    encoder_outputs = transformer_encoder(encoder_inputs, head_size, num_heads, ff_dim, dropout_rate,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

    # Decoder also uses the same sequence data for simplicity
    decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
    decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate)

    x = layers.GlobalAveragePooling1D()(encoder_outputs)
    outputs = layers.Dense(1)(x)

    model = keras.Model(inputs=encoder_inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

# Modify the `objective` function to use the correct data structure
def objective(trial):
    model = build_model(trial)

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train the model
    model.fit(X_train_seq, y_train_seq, epochs=30, batch_size=128,
          validation_data=(X_val_seq, y_val_seq),
          callbacks=[early_stopping, reduce_lr], verbose=1)

    # Get predictions on the validation set
    y_pred_scaled = model.predict([X_val_seq])

    # Calculate metrics on the scaled values
    mse_scaled = mean_squared_error(y_val_seq, y_pred_scaled)
    rmse_scaled = np.sqrt(mse_scaled)

    return rmse_scaled

# Create an Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Get the best trial and print the best hyperparameters
print(f"Best trial: {study.best_trial.number}")
print(f"Best hyperparameters: {study.best_trial.params}")

# Build the model using the best hyperparameters
best_model = build_model(study.best_trial)

early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Retrain the model with the best hyperparameters
best_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=128,
               validation_data=(X_val_seq, y_val_seq),
               callbacks=[early_stopping, reduce_lr])

# Get predictions on the test set
y_pred_scaled = best_model.predict([X_test_seq])

# Calculate metrics on the scaled values
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()



## 12.4.1 Transformer Optimiser - Short Term

In [ ]:
# List of optimizers to test
optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop,
    "adagrad": Adagrad,
    "adamax": Adamax,
    "nadam": Nadam
}

# Dictionary to store results
results = {}

# Load the dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=28, num_heads=6, ff_dim=79, dropout_rate=0.3397155165626943,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=28, num_heads=6, ff_dim=79, dropout_rate=0.3397155165626943)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Iterate over each optimizer
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    print(f"Using optimizer: {opt_name}")

    # Create the model
    model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

    model.compile(optimizer=opt_name, loss='mse', metrics=['mae'])

    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train the model
    history = model.fit(
        [X_train_seq, X_train_seq], y_train_seq,
        epochs=50,
        batch_size=67,
        validation_data=([X_val_seq, X_val_seq], y_val_seq),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Evaluate the model on the test set
    test_loss = model.evaluate([X_test_seq, X_test_seq], y_test_seq, verbose=1)  # MSE on test data
    
    # Get predictions on the test set
    y_pred_scaled = model.predict([X_test_seq, X_test_seq])

    # Calculate metrics on the scaled values
    mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
    rmse_scaled = np.sqrt(mse_scaled)
    mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

    # Store results
    results[opt_name] = {
        "Test Loss (MSE)": test_loss,
        "RMSE (scaled)": rmse_scaled,
        "MAE (scaled)": mae_scaled
    }

# Display results for comparison
print("\nOptimizer Performance Comparison:")
for opt_name, metrics in results.items():
    print(f"{opt_name}: {metrics}")

# Example Plot (Using the best optimizer's predictions)
best_optimizer = min(results, key=lambda k: results[k]["RMSE (scaled)"])
print(f"\nBest optimizer based on RMSE: {best_optimizer}")

# Re-train and visualize using the best optimizer
model.compile(optimizer=best_optimizer, loss='mse')
model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

final_predictions_scaled = model.predict([X_test_seq, X_test_seq])
final_predictions = target_scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(final_predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/Trans_STG_optimiser.png')
plt.show()



## 12.4.2 Transformer Optimiser - Mid Term

In [ ]:
# List of optimizers to test
optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop,
    "adagrad": Adagrad,
    "adamax": Adamax,
    "nadam": Nadam
}

# Dictionary to store results
results = {}

# Load the dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)

# Split the data into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Initialize scalers for features and target
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler on the training data and transform
X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

# Scale the target variable using the target scaler
y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sequences
def create_sequences(data, target, sequence_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        targets.append(target[i+sequence_length])
    return np.array(sequences), np.array(targets)

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length=5)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length=5)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length=5)

# Positional Encoding Function
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads[:, 0::2]), np.cos(angle_rads[:, 1::2])], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer Encoder block with Positional Encoding
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, seq_len, d_model):
    pos_encoding = get_positional_encoding(seq_len, d_model)
    inputs += pos_encoding
    
    attention = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention = layers.LayerNormalization(epsilon=1e-6)(attention + inputs)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention)

# Define Transformer Decoder block
def transformer_decoder(inputs, encoder_outputs, head_size, num_heads, ff_dim, dropout_rate):
    attention_1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs)
    
    attention_1 = layers.LayerNormalization(epsilon=1e-6)(attention_1 + inputs)
    
    attention_2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(attention_1, encoder_outputs)
    
    attention_2 = layers.LayerNormalization(epsilon=1e-6)(attention_2 + attention_1)
    
    ff = layers.Dense(ff_dim, activation='relu')(attention_2)
    ff = layers.Dense(inputs.shape[-1])(ff)
    
    return layers.LayerNormalization(epsilon=1e-6)(ff + attention_2)

# Define the Input Layer for Encoder (3D input: samples, time_steps, features)
encoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
encoder_outputs = transformer_encoder(encoder_inputs, head_size=37, num_heads=6, ff_dim=98, dropout_rate=0.11543766121575369,
                                      seq_len=X_train_seq.shape[1], d_model=X_train_seq.shape[2])

# Define the Input Layer for Decoder (3D input: samples, time_steps, features)
decoder_inputs = layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))  # (time_steps, features)
decoder_outputs = transformer_decoder(decoder_inputs, encoder_outputs, head_size=28, num_heads=6, ff_dim=79, dropout_rate=0.3397155165626943)

# Global Average Pooling to reduce the time dimension
x = layers.GlobalAveragePooling1D()(decoder_outputs)

# Output layer (for regression, a single neuron)
outputs = layers.Dense(1)(x)

# Iterate over each optimizer
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    print(f"Using optimizer: {opt_name}")

    # Create the model
    model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)

    model.compile(optimizer=opt_name, loss='mse', metrics=['mae'])

    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    # Train the model
    history = model.fit(
        [X_train_seq, X_train_seq], y_train_seq,
        epochs=50,
        batch_size=72,
        validation_data=([X_val_seq, X_val_seq], y_val_seq),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Evaluate the model on the test set
    test_loss = model.evaluate([X_test_seq, X_test_seq], y_test_seq, verbose=1)  # MSE on test data
    
    # Get predictions on the test set
    y_pred_scaled = model.predict([X_test_seq, X_test_seq])

    # Calculate metrics on the scaled values
    mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
    rmse_scaled = np.sqrt(mse_scaled)
    mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

    # Store results
    results[opt_name] = {
        "Test Loss (MSE)": test_loss,
        "RMSE (scaled)": rmse_scaled,
        "MAE (scaled)": mae_scaled
    }

# Display results for comparison
print("\nOptimizer Performance Comparison:")
for opt_name, metrics in results.items():
    print(f"{opt_name}: {metrics}")

# Example Plot (Using the best optimizer's predictions)
best_optimizer = min(results, key=lambda k: results[k]["RMSE (scaled)"])
print(f"\nBest optimizer based on RMSE: {best_optimizer}")

# Re-train and visualize using the best optimizer
model.compile(optimizer=best_optimizer, loss='mse')
model.fit(
    [X_train_seq, X_train_seq], y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=([X_val_seq, X_val_seq], y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

final_predictions_scaled = model.predict([X_test_seq, X_test_seq])
final_predictions = target_scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(final_predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/Trans_STG_optimiser.png')
plt.show()



## 12.5.1 Hybrid Model Hyperparameters - Short Term

In [ ]:
# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])  # This might be an issue
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Objective function for Optuna
def objective(trial):
    # Hyperparameter Search Space
    lstm_units = trial.suggest_int('lstm_units', 32, 256)
    head_size = trial.suggest_int('head_size', 32, 128)
    num_heads = trial.suggest_int('num_heads', 4, 12)
    ff_dim = trial.suggest_int('ff_dim', 128, 512)
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    
    # Build the model with the suggested hyperparameters
    input_shape = (sequence_length, features.shape[1])
    model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    
    # Train the model
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=30,
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        verbose = 1  
    )
    
    # Get predictions on the validation set
    y_pred_scaled = model.predict(X_val_seq)
    
    # Calculate MSE (Mean Squared Error) on the validation set
    mse = mean_squared_error(y_val_seq, y_pred_scaled)
    return mse  # Return the MSE to Optuna for optimization

# Create an Optuna study to minimize the objective (MSE)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Number of trials to run

# Print the best hyperparameters found by Optuna
print("Best Hyperparameters:")
print(study.best_params)

input_shape = (sequence_length, features.shape[1])

# You can now use the best parameters to train a final model:
best_params = study.best_params
final_model = build_model(
    input_shape, 
    best_params['lstm_units'],
    best_params['head_size'],
    best_params['num_heads'],
    best_params['ff_dim'],
    best_params['num_blocks'],
    best_params['dropout_rate']
)

# Final model compilation and training
final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mse', metrics=['mae'])
final_history = final_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=best_params['batch_size'],
    callbacks=[early_stopping, reduce_lr],
    verbose = 1
)

# Get predictions on the test set
y_pred_scaled = final_model.predict(X_test_seq)
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 12.5.2 Hybrid Model Hyperparameters - Mid Term

In [ ]:
# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_weekly_energy_consumption'])
target = stf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])  # This might be an issue
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

# Objective function for Optuna
def objective(trial):
    # Hyperparameter Search Space
    lstm_units = trial.suggest_int('lstm_units', 32, 256)
    head_size = trial.suggest_int('head_size', 32, 128)
    num_heads = trial.suggest_int('num_heads', 4, 12)
    ff_dim = trial.suggest_int('ff_dim', 128, 512)
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    
    # Build the model with the suggested hyperparameters
    input_shape = (sequence_length, features.shape[1])
    model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    
    # Train the model
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=30,
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        verbose = 1  
    )
    
    # Get predictions on the validation set
    y_pred_scaled = model.predict(X_val_seq)
    
    # Calculate MSE (Mean Squared Error) on the validation set
    mse = mean_squared_error(y_val_seq, y_pred_scaled)
    return mse  # Return the MSE to Optuna for optimization

# Create an Optuna study to minimize the objective (MSE)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Number of trials to run

# Print the best hyperparameters found by Optuna
print("Best Hyperparameters:")
print(study.best_params)

input_shape = (sequence_length, features.shape[1])

# You can now use the best parameters to train a final model:
best_params = study.best_params
final_model = build_model(
    input_shape, 
    best_params['lstm_units'],
    best_params['head_size'],
    best_params['num_heads'],
    best_params['ff_dim'],
    best_params['num_blocks'],
    best_params['dropout_rate']
)

# Final model compilation and training
final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mse', metrics=['mae'])
final_history = final_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=best_params['batch_size'],
    callbacks=[early_stopping, reduce_lr],
    verbose = 1
)

# Get predictions on the test set
y_pred_scaled = final_model.predict(X_test_seq)
mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
rmse_scaled = np.sqrt(mse_scaled)
mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

print(f"Mean Squared Error (MSE) on Scaled Data: {mse_scaled}")
print(f"Root Mean Squared Error (RMSE) on Scaled Data: {rmse_scaled}")
print(f"Mean Absolute Error (MAE) on Scaled Data: {mae_scaled}")

# Reverse the scaling for predictions and actual values
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/Hybrid_MTG_daily.png')
plt.show()


## 12.6.1 Hybrid Model Optimiser - Short Term

In [ ]:
# List of optimizers to test
optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop,
    "adagrad": Adagrad,
    "adamax": Adamax,
    "nadam": Nadam
}

# Dictionary to store results
results = {}

# Load dataset
stf = pd.read_csv('/home/u894059/.local/dataset/short_term.csv')

# Select the features and target variable
features = stf.drop(columns=['total_daily_energy_consumption'])
target = stf['total_daily_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

lstm_units = 130
head_size = 66
num_heads = 10
ff_dim = 133
num_blocks = 3
dropout_rate = 0.11640596573903705
learning_rate = 0.00039155660875927047
batch_size = 55

# Iterate over each optimizer
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    print(f"Using optimizer: {opt_name}")

    input_shape = (sequence_length, features.shape[1])
    model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)
    
    model.compile(optimizer=opt_name, loss='mse', metrics=['mae'])
    
    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
    
    # Train the model
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=50,
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        verbose=1 
    )
    # Evaluate the model on the test set
    test_loss = model.evaluate(X_test_seq, y_test_seq, verbose=1)

    # Get predictions on the validation set
    y_pred_scaled = model.predict(X_test_seq)
    
    # Calculate metrics on the scaled values
    mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
    rmse_scaled = np.sqrt(mse_scaled)
    mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

    # Store results
    results[opt_name] = {
        "Test Loss (MSE)": test_loss,
        "RMSE (scaled)": rmse_scaled,
        "MAE (scaled)": mae_scaled
    }

# Display results for comparison
print("\nOptimizer Performance Comparison:")
for opt_name, metrics in results.items():
    print(f"{opt_name}: {metrics}")

# Example Plot (Using the best optimizer's predictions)
best_optimizer = min(results, key=lambda k: results[k]["RMSE (scaled)"])
print(f"\nBest optimizer based on RMSE: {best_optimizer}")

# Re-train and visualize using the best optimizer
model.compile(optimizer=best_optimizer, loss='mse')
model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

final_predictions_scaled = model.predict(X_test_seq)
final_predictions = target_scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(final_predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()


## 12.6.2 Hybrid Model Optimiser - Mid Term

In [ ]:
# List of optimizers to test
optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop,
    "adagrad": Adagrad,
    "adamax": Adamax,
    "nadam": Nadam
}

# Dictionary to store results
results = {}

# Load dataset
mtf = pd.read_csv('/home/u894059/.local/dataset/mid_term.csv')

# Select the features and target variable
features = mtf.drop(columns=['total_weekly_energy_consumption'])
target = mtf['total_weekly_energy_consumption'].values.reshape(-1, 1)  # reshape for scaling

# Preprocess the data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, shuffle=False)

# Normalize the data
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = feature_scaler.fit_transform(X_train)
X_val_scaled = feature_scaler.transform(X_val)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train).flatten()
y_val_scaled = target_scaler.transform(y_val).flatten()
y_test_scaled = target_scaler.transform(y_test).flatten()

# Create sliding window sequences
def create_sequences(data, target, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(target[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Experiment with longer sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length)

# LSTM Block
def lstm_block(inputs, lstm_units):
    x = layers.LSTM(lstm_units, return_sequences=True, kernel_regularizer='l2')(inputs)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    return x

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate, num_blocks):
    for _ in range(num_blocks):
        # Multi-Head Attention
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual Connection
        
        # Feedforward Block
        ff = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
        ff = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual Connection
    
    return x

# Build the RS-LSTM-Transformer Model
def build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = layers.Input(shape=input_shape)
    
    # LSTM Block
    lstm_out = lstm_block(inputs, lstm_units)
    
    # Transformer Encoder
    transformer_out = transformer_encoder(lstm_out, head_size, num_heads, ff_dim, dropout_rate, num_blocks)
    
    # Global Average Pooling + Dense Layers
    gap = layers.GlobalAveragePooling1D()(transformer_out)
    dense = layers.Dense(64, activation='relu')(gap)
    outputs = layers.Dense(1)(dense)
    
    model = Model(inputs, outputs)
    return model

lstm_units = 98
head_size = 86
num_heads = 9
ff_dim = 134
num_blocks = 4
dropout_rate = 0.42901614838022517
learning_rate = 2.018835159150557e-05
batch_size = 17

# Iterate over each optimizer
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    print(f"Using optimizer: {opt_name}")

    input_shape = (sequence_length, features.shape[1])
    model = build_model(input_shape, lstm_units, head_size, num_heads, ff_dim, num_blocks, dropout_rate)
    
    model.compile(optimizer=opt_name, loss='mse', metrics=['mae'])
    
    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
    
    # Train the model
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=50,
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        verbose=1 
    )
    # Evaluate the model on the test set
    test_loss = model.evaluate(X_test_seq, y_test_seq, verbose=1)

    # Get predictions on the validation set
    y_pred_scaled = model.predict(X_test_seq)
    
    # Calculate metrics on the scaled values
    mse_scaled = mean_squared_error(y_test_seq, y_pred_scaled)
    rmse_scaled = np.sqrt(mse_scaled)
    mae_scaled = mean_absolute_error(y_test_seq, y_pred_scaled)

    # Store results
    results[opt_name] = {
        "Test Loss (MSE)": test_loss,
        "RMSE (scaled)": rmse_scaled,
        "MAE (scaled)": mae_scaled
    }

# Display results for comparison
print("\nOptimizer Performance Comparison:")
for opt_name, metrics in results.items():
    print(f"{opt_name}: {metrics}")

# Example Plot (Using the best optimizer's predictions)
best_optimizer = min(results, key=lambda k: results[k]["RMSE (scaled)"])
print(f"\nBest optimizer based on RMSE: {best_optimizer}")

# Re-train and visualize using the best optimizer
model.compile(optimizer=best_optimizer, loss='mse')
model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, reduce_lr]
)

final_predictions_scaled = model.predict(X_test_seq)
final_predictions = target_scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()
y_test_original = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

# Plot the actual vs predicted values
plt.plot(y_test_original, label='Actual')
plt.plot(final_predictions, label='Predicted')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Energy Consumption')
plt.title('Actual vs Predicted Energy Consumption')
plt.savefig('/home/u894059/.local/plot1.png')
plt.show()
